In [ ]:
#problem statement:
#scope: combine location wise bedplaces and percentage of individuals and get location wise bedplaces per individual

import pandas as pd

# Read the CSV file into a pandas DataFrame
# location wise bedplaces etc data
df = pd.read_csv('data/estat_tour_cap_nat_en.csv')

#location wise percentage of individuals
df_2 = pd.read_csv('data/estat_isoc_ci_dev_i_en.csv')

In [10]:
df.head()

,DATAFLOW,LAST UPDATE,freq,accomunit,unit,nace_r2,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:TOUR_CAP_NAT(1.0),10/04/25 23:00:00,A,BEDPL,NR,I551,AT,1990,650559.0,NaN,NaN
1,ESTAT:TOUR_CAP_NAT(1.0),10/04/25 23:00:00,A,BEDPL,NR,I551,AT,1991,654127.0,NaN,NaN
2,ESTAT:TOUR_CAP_NAT(1.0),10/04/25 23:00:00,A,BEDPL,NR,I551,AT,1992,653586.0,NaN,NaN
3,ESTAT:TOUR_CAP_NAT(1.0),10/04/25 23:00:00,A,BEDPL,NR,I551,AT,1993,651003.0,NaN,NaN
4,ESTAT:TOUR_CAP_NAT(1.0),10/04/25 23:00:00,A,BEDPL,NR,I551,AT,1994,650020.0,NaN,NaN


In [11]:
# columns to use: “accomunit,unit,nace_r2,geo” and “2016 “
columns = [
    'accomunit',
    'unit',
    'nace_r2',
    'geo',
    'TIME_PERIOD',
    'OBS_VALUE'
]
# Filter the DataFrame to only include the specified columns
df = df[columns]


In [12]:
#rows to use: “accomunit” is “BEDPL”, “unit” is “NR”, “nace_r2” is “I551”, “TIME_PERDIO” is “2016”
df = df[
    (df['accomunit'] == 'BEDPL') &
    (df['unit'] == 'NR') &
    (df['nace_r2'] == 'I551') &
    (df['TIME_PERIOD'] == 2016)
]

In [16]:
df_2.head()

,DATAFLOW,LAST UPDATE,freq,ind_type,indic_is,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:ISOC_CI_DEV_I(1.0),17/12/24 11:00:00,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AL,2018,NaN,u,NaN
1,ESTAT:ISOC_CI_DEV_I(1.0),17/12/24 11:00:00,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT,2016,37.41,NaN,NaN
2,ESTAT:ISOC_CI_DEV_I(1.0),17/12/24 11:00:00,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT,2018,42.45,NaN,NaN
3,ESTAT:ISOC_CI_DEV_I(1.0),17/12/24 11:00:00,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT,2021,39.62,NaN,NaN
4,ESTAT:ISOC_CI_DEV_I(1.0),17/12/24 11:00:00,A,CB_EU_FOR,I_IUG_DKPC,PC_IND,AT,2023,40.34,NaN,NaN


In [17]:
# Rows to use: “ind_type” is “IND_TOTAL”, “indic_is” is “I_IUG_TV”, “unit” is “PC_IND”, “TIME_PERIOD” is “2016”
df_2 = df_2[
    (df_2['ind_type'] == 'IND_TOTAL') &
    (df_2['indic_is'] == 'I_IUG_TV') &
    (df_2['unit'] == 'PC_IND') &
    (df_2['TIME_PERIOD'] == 2016)
]

In [18]:
# Remove specific regions from the 'geo' column
regions_to_remove = ["EA", "EU27_2007", "EU27_2020", "EU28", "EA19"]
df = df[~df['geo'].isin(regions_to_remove)]
df_2 = df_2[~df_2['geo'].isin(regions_to_remove)]